# Vietnam Traffic Detection - YOLOv8 Training on Kaggle

This notebook trains a custom YOLOv8 model for Vietnam traffic detection.

**Classes:** car, truck, motorcycle, bus, ambulance (5 classes)

**Requirements:**
- Kaggle GPU runtime (P100 or T4)
- Dataset uploaded to Kaggle Datasets

**Estimated Training Time:** 2-4 hours for 100 epochs

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install ultralytics>=8.0.0 -q
!pip install albumentations>=1.3.0 -q

# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
import os
import shutil
from pathlib import Path
from ultralytics import YOLO
import yaml

# Set working directory
WORK_DIR = Path('/kaggle/working')
os.chdir(WORK_DIR)
print(f"Working directory: {WORK_DIR}")

## 2. Dataset Configuration

**IMPORTANT:** Before running this cell, upload your dataset to Kaggle:
1. Go to kaggle.com/datasets/new
2. Upload your labeled dataset (in YOLO format)
3. Note the dataset path (e.g., `/kaggle/input/vietnam-traffic-dataset`)
4. Update `DATASET_PATH` below

In [ ]:
# ============================================================
# CONFIGURE YOUR DATASET PATH HERE
# ============================================================
DATASET_PATH = '/kaggle/input/vietnam-traffic-dataset'  # <-- UPDATE THIS!
# ============================================================

# Verify dataset exists
if not os.path.exists(DATASET_PATH):
    print(f"ERROR: Dataset not found at {DATASET_PATH}")
    print("Please upload your dataset to Kaggle and update DATASET_PATH")
else:
    print(f"Dataset found at: {DATASET_PATH}")
    print("Contents:")
    for item in os.listdir(DATASET_PATH):
        item_path = os.path.join(DATASET_PATH, item)
        if os.path.isdir(item_path):
            count = len(os.listdir(item_path))
            print(f"  {item}/ ({count} items)")
        else:
            print(f"  {item}")

In [ ]:
# Create dataset.yaml for training
# This maps to your uploaded dataset structure

dataset_config = {
    'path': DATASET_PATH,
    'train': 'images/train',
    'val': 'images/val',
    'test': 'images/test',
    'nc': 5,
    'names': {
        0: 'car',
        1: 'truck',
        2: 'motorcycle',
        3: 'bus',
        4: 'ambulance'
    }
}

# Save dataset.yaml
dataset_yaml_path = WORK_DIR / 'dataset.yaml'
with open(dataset_yaml_path, 'w') as f:
    yaml.dump(dataset_config, f, default_flow_style=False)

print(f"Dataset config saved to: {dataset_yaml_path}")
print("\nConfig contents:")
with open(dataset_yaml_path, 'r') as f:
    print(f.read())

In [ ]:
# Verify dataset structure
def count_files(directory, extension='*'):
    if not os.path.exists(directory):
        return 0
    if extension == '*':
        return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])
    return len([f for f in os.listdir(directory) if f.endswith(extension)])

print("Dataset Statistics:")
print("="*40)

for split in ['train', 'val', 'test']:
    img_dir = os.path.join(DATASET_PATH, 'images', split)
    lbl_dir = os.path.join(DATASET_PATH, 'labels', split)
    
    n_images = count_files(img_dir)
    n_labels = count_files(lbl_dir, '.txt')
    
    print(f"{split.capitalize():8s}: {n_images:4d} images, {n_labels:4d} labels")
    
    if n_images != n_labels:
        print(f"  WARNING: Image/label count mismatch!")

## 3. Training Configuration

Optimized settings for Vietnam traffic detection:
- YOLOv8l (large) for best accuracy
- Transfer learning from COCO pretrained weights
- Augmentation tuned for traffic camera footage

In [ ]:
# ============================================================
# TRAINING HYPERPARAMETERS - Adjust as needed
# ============================================================

TRAINING_CONFIG = {
    # Model selection
    'model': 'yolov8l.pt',       # Options: yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
    
    # Training parameters
    'epochs': 100,               # Number of training epochs
    'batch': 16,                 # Batch size (reduce if OOM)
    'imgsz': 640,                # Image size (640 is standard)
    'patience': 20,              # Early stopping patience
    
    # Learning rate
    'lr0': 0.01,                 # Initial learning rate
    'lrf': 0.01,                 # Final learning rate (lr0 * lrf)
    'warmup_epochs': 3,          # Warmup epochs
    
    # Optimizer
    'optimizer': 'SGD',          # Optimizer (SGD, Adam, AdamW)
    'momentum': 0.937,           # SGD momentum
    'weight_decay': 0.0005,      # Weight decay
    
    # Augmentation (tuned for traffic cameras)
    'hsv_h': 0.015,              # HSV-Hue augmentation
    'hsv_s': 0.7,                # HSV-Saturation
    'hsv_v': 0.4,                # HSV-Value (brightness)
    'degrees': 0.0,              # Rotation (disabled for traffic)
    'translate': 0.1,            # Translation
    'scale': 0.5,                # Scale
    'shear': 0.0,                # Shear (disabled)
    'perspective': 0.0,          # Perspective (disabled)
    'flipud': 0.0,               # Vertical flip (disabled)
    'fliplr': 0.5,               # Horizontal flip
    'mosaic': 1.0,               # Mosaic augmentation
    'mixup': 0.0,                # MixUp (disabled)
    
    # Other settings
    'workers': 4,                # DataLoader workers
    'device': 0,                 # GPU device (0 for single GPU)
    'project': 'vietnam_traffic',
    'name': 'yolov8l_custom',
    'exist_ok': True,
    'pretrained': True,
    'verbose': True,
    'seed': 42,
}

print("Training Configuration:")
for k, v in TRAINING_CONFIG.items():
    print(f"  {k}: {v}")

## 4. Model Training

In [ ]:
# Load pretrained YOLOv8 model
model = YOLO(TRAINING_CONFIG['model'])

print(f"Model loaded: {TRAINING_CONFIG['model']}")
print(f"Model type: {type(model)}")

In [ ]:
# Start training!
# This cell will take 2-4 hours depending on your dataset size and GPU

results = model.train(
    data=str(dataset_yaml_path),
    epochs=TRAINING_CONFIG['epochs'],
    batch=TRAINING_CONFIG['batch'],
    imgsz=TRAINING_CONFIG['imgsz'],
    patience=TRAINING_CONFIG['patience'],
    
    # Learning rate
    lr0=TRAINING_CONFIG['lr0'],
    lrf=TRAINING_CONFIG['lrf'],
    warmup_epochs=TRAINING_CONFIG['warmup_epochs'],
    
    # Optimizer
    optimizer=TRAINING_CONFIG['optimizer'],
    momentum=TRAINING_CONFIG['momentum'],
    weight_decay=TRAINING_CONFIG['weight_decay'],
    
    # Augmentation
    hsv_h=TRAINING_CONFIG['hsv_h'],
    hsv_s=TRAINING_CONFIG['hsv_s'],
    hsv_v=TRAINING_CONFIG['hsv_v'],
    degrees=TRAINING_CONFIG['degrees'],
    translate=TRAINING_CONFIG['translate'],
    scale=TRAINING_CONFIG['scale'],
    shear=TRAINING_CONFIG['shear'],
    perspective=TRAINING_CONFIG['perspective'],
    flipud=TRAINING_CONFIG['flipud'],
    fliplr=TRAINING_CONFIG['fliplr'],
    mosaic=TRAINING_CONFIG['mosaic'],
    mixup=TRAINING_CONFIG['mixup'],
    
    # Other
    workers=TRAINING_CONFIG['workers'],
    device=TRAINING_CONFIG['device'],
    project=TRAINING_CONFIG['project'],
    name=TRAINING_CONFIG['name'],
    exist_ok=TRAINING_CONFIG['exist_ok'],
    pretrained=TRAINING_CONFIG['pretrained'],
    verbose=TRAINING_CONFIG['verbose'],
    seed=TRAINING_CONFIG['seed'],
)

print("\nTraining completed!")

## 5. Evaluate Trained Model

In [ ]:
# Find the best model weights
best_model_path = Path(TRAINING_CONFIG['project']) / TRAINING_CONFIG['name'] / 'weights' / 'best.pt'
last_model_path = Path(TRAINING_CONFIG['project']) / TRAINING_CONFIG['name'] / 'weights' / 'last.pt'

print(f"Best model: {best_model_path}")
print(f"Last model: {last_model_path}")
print(f"\nBest model exists: {best_model_path.exists()}")

In [ ]:
# Load the best model and run validation
best_model = YOLO(str(best_model_path))

# Validate on validation set
val_results = best_model.val(
    data=str(dataset_yaml_path),
    split='val',
    batch=TRAINING_CONFIG['batch'],
    imgsz=TRAINING_CONFIG['imgsz'],
    verbose=True,
)

In [ ]:
# Print validation metrics
print("\n" + "="*50)
print("VALIDATION RESULTS")
print("="*50)

print(f"\nmAP50:     {val_results.box.map50:.4f}")
print(f"mAP50-95:  {val_results.box.map:.4f}")

print("\nPer-class AP50:")
class_names = ['car', 'truck', 'motorcycle', 'bus', 'ambulance']
for i, name in enumerate(class_names):
    if i < len(val_results.box.ap50):
        print(f"  {name:12s}: {val_results.box.ap50[i]:.4f}")

In [ ]:
# Test on test set (if available)
test_dir = os.path.join(DATASET_PATH, 'images', 'test')
if os.path.exists(test_dir) and len(os.listdir(test_dir)) > 0:
    print("Running evaluation on test set...")
    test_results = best_model.val(
        data=str(dataset_yaml_path),
        split='test',
        batch=TRAINING_CONFIG['batch'],
        imgsz=TRAINING_CONFIG['imgsz'],
        verbose=True,
    )
    print(f"\nTest mAP50:    {test_results.box.map50:.4f}")
    print(f"Test mAP50-95: {test_results.box.map:.4f}")
else:
    print("No test set found, skipping test evaluation.")

## 6. Visualize Results

In [ ]:
# Display training curves and confusion matrix
from IPython.display import Image, display

results_dir = Path(TRAINING_CONFIG['project']) / TRAINING_CONFIG['name']

# Training curves
results_png = results_dir / 'results.png'
if results_png.exists():
    print("Training Curves:")
    display(Image(filename=str(results_png), width=800))

In [ ]:
# Confusion matrix
confusion_matrix_png = results_dir / 'confusion_matrix.png'
if confusion_matrix_png.exists():
    print("Confusion Matrix:")
    display(Image(filename=str(confusion_matrix_png), width=600))

In [ ]:
# Sample predictions on validation images
val_batch_png = results_dir / 'val_batch0_pred.jpg'
if val_batch_png.exists():
    print("Sample Predictions:")
    display(Image(filename=str(val_batch_png), width=800))

## 7. Export Model

Export the trained model in various formats for deployment.

In [ ]:
# Export to ONNX for cross-platform deployment
onnx_path = best_model.export(format='onnx', imgsz=640, simplify=True)
print(f"ONNX model exported to: {onnx_path}")

In [ ]:
# Export to TorchScript for PyTorch deployment
torchscript_path = best_model.export(format='torchscript', imgsz=640)
print(f"TorchScript model exported to: {torchscript_path}")

## 8. Download Trained Model

Copy the trained model to the output directory for download.

In [ ]:
# Create output directory for download
output_dir = WORK_DIR / 'trained_model'
output_dir.mkdir(exist_ok=True)

# Copy best model
shutil.copy(best_model_path, output_dir / 'vietnam_traffic_yolov8l_best.pt')
shutil.copy(last_model_path, output_dir / 'vietnam_traffic_yolov8l_last.pt')

# Copy ONNX if exists
onnx_file = best_model_path.with_suffix('.onnx')
if onnx_file.exists():
    shutil.copy(onnx_file, output_dir / 'vietnam_traffic_yolov8l.onnx')

# Copy results
if results_png.exists():
    shutil.copy(results_png, output_dir / 'training_results.png')
if confusion_matrix_png.exists():
    shutil.copy(confusion_matrix_png, output_dir / 'confusion_matrix.png')

print(f"\nTrained model files copied to: {output_dir}")
print("\nFiles available for download:")
for f in output_dir.iterdir():
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f"  {f.name}: {size_mb:.1f} MB")

In [ ]:
# Create a zip file for easy download
import zipfile

zip_path = WORK_DIR / 'vietnam_traffic_model.zip'
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in output_dir.iterdir():
        zipf.write(file, file.name)

zip_size_mb = zip_path.stat().st_size / (1024 * 1024)
print(f"\nZip file created: {zip_path}")
print(f"Size: {zip_size_mb:.1f} MB")
print("\nDownload this file from the Output tab in Kaggle!")

## 9. Quick Test Inference

Test the trained model on a sample image.

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Find a sample image from validation set
val_images_dir = Path(DATASET_PATH) / 'images' / 'val'
if val_images_dir.exists():
    sample_images = list(val_images_dir.glob('*.jpg'))[:3]
    
    if sample_images:
        fig, axes = plt.subplots(1, len(sample_images), figsize=(15, 5))
        if len(sample_images) == 1:
            axes = [axes]
        
        for ax, img_path in zip(axes, sample_images):
            # Run inference
            results = best_model(str(img_path), verbose=False)
            
            # Plot result
            result_img = results[0].plot()
            result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
            
            ax.imshow(result_img)
            ax.set_title(img_path.name)
            ax.axis('off')
        
        plt.tight_layout()
        plt.savefig(output_dir / 'sample_predictions.png', dpi=150)
        plt.show()
        
        print(f"\nSample predictions saved to: {output_dir / 'sample_predictions.png'}")

## 10. Integration Instructions

After training, follow these steps to integrate the model:

1. **Download** `vietnam_traffic_model.zip` from Kaggle Output

2. **Extract** and copy `vietnam_traffic_yolov8l_best.pt` to your project's `video_detection/` folder

3. **Update** `video_detection/config/config.yaml`:
```yaml
model:
  path: "vietnam_traffic_yolov8l_best.pt"  # Changed from yolov8l.pt
```

4. **Update** `video_detection/detector/yolo_detector.py` class mapping (if needed):
```python
# The trained model uses these class indices:
# 0: car, 1: truck, 2: motorcycle, 3: bus, 4: ambulance
```

5. **Test** the model:
```bash
cd video_detection
python main.py --video ../dataset/01.mp4 --model vietnam_traffic_yolov8l_best.pt
```

In [ ]:
# Print final summary
print("="*60)
print("TRAINING COMPLETE - SUMMARY")
print("="*60)
print(f"\nModel: {TRAINING_CONFIG['model']}")
print(f"Epochs: {TRAINING_CONFIG['epochs']}")
print(f"Image size: {TRAINING_CONFIG['imgsz']}")
print(f"\nValidation mAP50: {val_results.box.map50:.4f}")
print(f"Validation mAP50-95: {val_results.box.map:.4f}")
print(f"\nOutput files:")
print(f"  - vietnam_traffic_yolov8l_best.pt (main model)")
print(f"  - vietnam_traffic_yolov8l.onnx (ONNX export)")
print(f"  - vietnam_traffic_model.zip (all files)")
print(f"\nDownload from Kaggle Output tab!")